In [ ]:
from ipydatagrid import DataGrid, Filter, Sort

In [ ]:
from json import load

with open('./cars.json') as fobj:  
    data = load(fobj)

In [ ]:
from bqplot import ColorScale, OrdinalColorScale

formatters = {
    'Acceleration': {'background_color': ColorScale(min=0, max=20)},
    'Year': {'background_color': 'green'},
    'Displacement': {'text_color': ColorScale(min=97, max=455)},
    'Origin': {'background_color': OrdinalColorScale(
        colors=['DodgerBlue', 'SeaGreen', 'Yellow'], domain=['USA', 'Japan', 'Europe'])}
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, formatters=formatters)
datagrid

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])

# Perfomance test: A million cells with ipydatagrid

In [ ]:
from random import uniform

n_rows = 1000
n_columns = 1000

data = {'data': [], 'schema': {}}

data['data'] = [[uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)]
data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]

In [ ]:
from bqplot import ColorScale

color_scale = ColorScale(min=0, max=1, scheme='viridis')

formatters = {str(c): {'background_color': color_scale} for c in range(n_columns)}

huge_datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, formatters=formatters)

In [ ]:
huge_datagrid